In [18]:
SEASON = '2014-15'

In [19]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins
1,Stephen Curry,GSW,PG,19.000554,26,80,80,32.7,8.2,16.8,...,28.9,11.5,4.1,15.7,0.288,8.2,1.7,9.9,7.9,67.0
2,James Harden,HOU,SG,18.939508,25,81,81,36.8,8.0,18.1,...,31.3,12.2,4.2,16.4,0.265,7.0,1.8,8.8,8.1,56.0
3,Chris Paul,LAC,PG,16.606875,29,82,82,34.8,6.9,14.3,...,23.7,12.9,3.2,16.1,0.270,6.4,1.6,8.0,7.2,56.0
4,Anthony Davis,NOP,PF,15.977986,21,68,68,36.1,9.4,17.6,...,27.8,9.9,4.2,14.0,0.274,6.6,2.3,8.9,6.7,45.0
5,Russell Westbrook,OKC,PG,14.813247,26,67,67,34.4,9.4,22.0,...,38.4,7.5,3.2,10.6,0.222,6.9,1.2,8.1,5.8,45.0
6,LeBron James,CLE,SF,13.915491,30,69,69,36.1,9.0,18.5,...,32.3,7.4,2.9,10.4,0.199,6.1,1.0,7.1,5.7,53.0
7,Damian Lillard,POR,PG,10.830395,24,82,82,35.7,7.2,16.6,...,26.9,7.3,3.3,10.6,0.174,4.3,0.3,4.7,4.9,51.0
8,Marc Gasol,MEM,C,10.765434,30,81,81,33.2,6.5,13.2,...,24.6,5.5,4.7,10.2,0.182,2.7,2.4,5.1,4.8,55.0
9,Jimmy Butler,CHI,SG,10.085289,25,65,65,38.7,6.5,14.0,...,21.6,8.2,3.0,11.2,0.214,3.4,1.2,4.6,4.2,50.0
10,Klay Thompson,GSW,SG,10.074365,24,77,77,31.9,7.8,16.9,...,27.6,5.7,3.1,8.8,0.172,4.5,-0.1,4.4,4.0,67.0
